In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


In [2]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [3]:
memotion_data = pd.read_csv("data3.csv")

memotion_data .shape

(6595, 4)

In [4]:
# Here Sentiment Humor and offensive_or_not describe the meme
# For Sentiment: 
#  5 -> very_positive
#  4 -> positive
#  3 -> neutral
#  2 -> negative
#  1 -> very negative


# For Humor: 
#  5 -> non motivational
#  4 -> very twisted
#  3 -> not sarcastic
#  2 -> General
#  1 -> Twisted meaning


# For Offensive_or_not: 
#  5 -> motivational
#  4 -> not offensive
#  3 -> slight
#  2 -> hateful offensive
#  1 -> very offensive

memotion_data.head()

,data1,Sentiment,Humor,Offensive_or_not
0,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,5,2,4
1,The best of #10 YearChallenge! Completed in le...,5,2,4
2,Sam Thorne @Strippin ( Follow Follow Saw every...,4,3,4
3,10 Year Challenge - Sweet Dee Edition,4,1,1
4,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3,4,1


In [5]:
memotion_data.isnull().values.any()

True


In [6]:
memotion_data.dropna

<bound method DataFrame.dropna of                                                   data1  Sentiment  Humor  \
0     LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...          5      2   
1     The best of #10 YearChallenge! Completed in le...          5      2   
2     Sam Thorne @Strippin ( Follow Follow Saw every...          4      3   
3                10 Year Challenge - Sweet Dee Edition           4      1   
4     10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...          3      4   
5     1998: "Don't get in car with strangers" 2008: ...          2      2   
6     10 years challenge is lit af Üs & B erg 1945 1...          2      3   
7     10 Year Challenge emotional edition boredpanda...          3      1   
8     Fornite died in 10 months but Minecraft never ...          4      3   
9     FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...          4      2   
10    PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...          2      2   
11    State Dining Room in the White House

In [7]:
def preprocess(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus


In [8]:
memotion_data['data1']=preprocess(memotion_data.data1.values)
memotion_data.head()

In [9]:
#Checking polarity of sentement so dropping others
df_train=memotion_data.drop(columns=['Humor','Offensive_or_not'])
df_train.shape

(6595, 2)

In [10]:
df_train.head()

,data1,Sentiment
0,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,5
1,The best of #10 YearChallenge! Completed in le...,5
2,Sam Thorne @Strippin ( Follow Follow Saw every...,4
3,10 Year Challenge - Sweet Dee Edition,4
4,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3


In [11]:
print ("very_positive sentiments:", len(df_train[df_train.Sentiment == 5]))
print ("positive sentiments:", len(df_train[df_train.Sentiment == 4]))
print ("neutral sentiments:", len(df_train[df_train.Sentiment == 3]))
print ("negative sentiments:", len(df_train[df_train.Sentiment == 2]))
print ("very_negative sentiments:", len(df_train[df_train.Sentiment == 1]))

very_positive sentiments: 958
positive sentiments: 2912
neutral sentiments: 2073
negative sentiments: 439
very_negative sentiments: 138


In [12]:
df_test=memotion_data.drop(columns=['Sentiment','Humor','Offensive_or_not'])
df_test.shape


(6595, 1)

In [13]:
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy

In [14]:
train_text=df_train.data1.values
test_text=df_test.data1.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(6595,) (6595,) (6595, 6)


In [15]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)


(5276,) (5276, 6)
(1319,) (1319, 6)


In [16]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_text)
vocab_size = len(tokenizer.word_index) + 1
maxlen = 100

In [17]:
embeddings_dictionary = dict()

In [18]:
# Using glove word emdeddings
glove_file = open('glove.6B.100d.txt', encoding="utf8")

In [19]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [20]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [21]:
print(embedding_matrix.shape)

(11221, 100)


In [22]:
max_features = 10000
max_words = 100
batch_size = 128
epochs = 3
num_classes=5

In [23]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [24]:
from keras.preprocessing import sequence,text

In [25]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(5276, 100) (1319, 100) (6595, 100)


In [26]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_text)

X_train = tokenizer.texts_to_sequences(X_train_text)
X_test = tokenizer.texts_to_sequences(X_val_text)

In [27]:
from keras.layers import Dense,Dropout,Embedding,LSTM

In [28]:
from keras.optimizers import Adam

In [29]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(128))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1122100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,239,477
Trainable params: 117,377
Non-trainable params: 1,122,100
_________________________________________________________________


In [31]:
history1=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=128, verbose=1)

Train on 5276 samples, validate on 1319 samples
Epoch 1/3
5276/5276 [==============================] - 25s 1ms/step - loss: 1.0025 - acc: 0.5072 - val_loss: 0.8539 -
val_acc: 0.6494
Epoch 2/3
5276/5276 [==============================] - 26s 1ms/step - loss: 0.8024 - acc: 0.5699 - val_loss: 0.8116 -
val_acc: 0.6717
Epoch 3/3
5276/5276 [==============================] - 25s 1ms/step - loss: 0.7365 - acc: 0.6045 - val_loss: 0.8114 -
val_acc: 0.6713
CPU times: user 1min 16s, sys: 27.2 s, total: 1min 43s
Wall time: 51s
